# Evaluation Context for Rust - EvCxR

**EvCxR** = **Ev**aluation **C**onte**x**t for **R**ust

File: evcxr_demo_7.ipynb

Presentation to: Waikato Linux User Group

Author: Ian Stewart

Date: xx xxx 2019

Repository: https://github.com/WLUG/meetings/tree/master/2019/2019-xx-xx

EvCxR is a Juypter Kernel that allows Jupyter notebook to run the Rust programming language.

# Reading Files

Reading from files. Refer to:

https://stackoverflow.com/questions/31192956/whats-the-de-facto-way-of-reading-and-writing-files-in-rust-1-x


In [ ]:
// Rust 1.26 and onwards
// If you don't want to care about the underlying details, there are one-line functions for reading and writing.
// Read a file to a String
use std::fs;
main(); // For Jupyter only. Remove when using this code in a .rs file

pub fn main() {
    let data = fs::read_to_string("data_file_1").expect("Unable to read file");
    println!("{}", data);
}

In [ ]:
//Read a file as a Vec<u8>
use std::fs;
main(); // For Jupyter only. Remove when using this code in a .rs file

pub fn main() {
    let data = fs::read("data_file_1").expect("Unable to read file");
    println!("Length of data read is {} bytes", data.len());
    //println!("{}", data);  // `std::vec::Vec<u8>` cannot be formatted with the default formatter
    
    
    // If you have a vector of bytes (Vec<u8>) and want to convert to a String, 
    // the most efficient is to reuse the allocation with String::from_utf8:
    let s = String::from_utf8(data).expect("Found invalid UTF-8");
    println!("Data read from file: \n{}", s);
}

Rust 1.0 and onwards

These forms are slightly more verbose than the one-line functions that allocate a String or Vec for you, but are more powerful in that you can reuse allocated data or append to an existing object.

In [ ]:
// Read a file to a String
use std::fs::File;
use std::io::Read;
main(); // For Jupyter only. Remove when using this code in a .rs file

pub fn main() {
    let mut data = String::new();
    let mut f = File::open("/etc/hosts").expect("Unable to open file");
    f.read_to_string(&mut data).expect("Unable to read string");
    println!("{}", data);
}

In [ ]:
//Read a file as a Vec<u8>

use std::fs::File;
use std::io::Read;
main(); // For Jupyter only. Remove when using this code in a .rs file

pub fn main() {
    let mut data = Vec::new();
    let mut f = File::open("/etc/hosts").expect("Unable to open file");
    f.read_to_end(&mut data).expect("Unable to read data");
    println!("Length of data: {}", data.len());
}

Buffered I/O

    I felt a bit of a push from the community to use BufReader and BufWriter instead of reading straight from a file

A buffered reader (or writer) uses a buffer to reduce the number of I/O requests. For example, it's much more efficient to access the disk once to read 256 bytes instead of accessing the disk 256 times.

That being said, I don't believe a buffered reader/writer will be useful when reading the entire file. read_to_end seems to copy data in somewhat large chunks, so the transfer may already be naturally coalesced into fewer I/O requests.

Here's an example of using it for reading:

In [ ]:
use std::fs::File;
use std::io::{BufReader, Read};
main();  // For Jupyter only. Remove when using this code in a .rs file

pub fn main() {
    let mut data = String::new();
    let f = File::open("/etc/hosts").expect("Unable to open file");
    let mut br = BufReader::new(f);
    br.read_to_string(&mut data).expect("Unable to read string");
    println!("{}", data);
}

In [ ]:
//A BufReader is more useful when you want to read line-by-line:
use std::fs::File;
use std::io::{BufRead, BufReader};
main(); // For Jupyter only. Remove when using this code in a .rs file

pub fn main() {
    let f = File::open("/etc/hosts").expect("Unable to open file");
    let f = BufReader::new(f);

    for line in f.lines() {
        let line = line.expect("Unable to read line");
        println!("Line: {}", line);
    }
}